In [26]:
%matplotlib inline
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



import os
import numpy as np
import numpy.random as rand
from sklearn.ensemble import (GradientBoostingRegressor, 
                              GradientBoostingClassifier, 
                              AdaBoostClassifier,
                              RandomForestClassifier)
from sklearn.svm import SVC
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('bmh')

from pylab import rcParams
rcParams['figure.figsize'] = (8.0, 6.0)

from IPython.display import HTML

if not os.path.exists("./plots"):
    os.mkdir("./plots")


In [27]:
survey_place = pd.read_csv('caltrans_full_survey/survey_place.csv')

/home/g49/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
columns = ['arr_time', 'dep_time', 'trip_distance_miles', 'air_trip_distance_miles', 'prev_trip_duration_min', 'act_dur', 'act_cnt', 'tract_id', 'county_id', 'state_id', 'place_primarycity', 'city', 'state', 'zipcode', 'mode']
survey_place_less = survey_place[columns]
survey_place_less.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460524 entries, 0 to 460523
Data columns (total 15 columns):
arr_time                   460523 non-null object
dep_time                   460523 non-null object
trip_distance_miles        349385 non-null float64
air_trip_distance_miles    351745 non-null float64
prev_trip_duration_min     351745 non-null float64
act_dur                    460523 non-null float64
act_cnt                    460522 non-null float64
tract_id                   460519 non-null float64
county_id                  460519 non-null float64
state_id                   460523 non-null float64
place_primarycity          460520 non-null object
city                       460520 non-null object
state                      459795 non-null object
zipcode                    460523 non-null object
mode                       351745 non-null float64
dtypes: float64(9), object(6)
memory usage: 52.7+ MB


In [58]:
survey_place_less['arr_time'] = pd.to_datetime(survey_place_less['arr_time'], infer_datetime_format=True, format='%H:%M:%S').dt.hour
survey_place_less['dep_time'] = pd.to_datetime(survey_place_less['dep_time'], infer_datetime_format=True, format='%H:%M:%S').dt.hour

/home/g49/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/g49/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [59]:
survey_place_less.describe()

,arr_time,dep_time,trip_distance_miles,air_trip_distance_miles,prev_trip_duration_min,act_dur,act_cnt,tract_id,county_id,state_id,mode
count,460523.000000,460523.000000,349385.000000,351745.000000,351745.000000,460523.000000,460522.000000,460519.000000,460519.000000,460523.000000,351745.000000
mean,10.953898,10.516832,10.548866,8.516692,20.719905,324.336917,1.313119,191724.321311,58.057926,6.405207,5.256100
std,5.849350,6.079197,92.431734,80.018662,32.938191,376.087583,0.649834,242716.346473,50.000449,5.100262,3.171583
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,100.000000,1.000000,1.000000,1.000000
25%,6.000000,5.000000,1.045543,0.725180,7.000000,32.000000,1.000000,5911.000000,37.000000,6.000000,5.000000
50%,11.000000,11.000000,2.824188,2.019782,14.000000,218.000000,1.000000,43317.000000,59.000000,6.000000,5.000000
75%,16.000000,16.000000,7.824843,5.665627,24.000000,490.000000,1.000000,402800.000000,79.000000,6.000000,6.000000
max,23.000000,23.000000,17228.678974,8510.354989,1080.000000,26330.000000,3.000000,999999.000000,999.000000,99.000000,29.000000


In [60]:
survey_place_less.isnull().sum()

arr_time                        1
dep_time                        1
trip_distance_miles        111139
air_trip_distance_miles    108779
prev_trip_duration_min     108779
act_dur                         1
act_cnt                         2
tract_id                        5
county_id                       5
state_id                        1
place_primarycity               4
city                            4
state                         729
zipcode                         1
mode                       108779
dtype: int64

In [61]:
def get_percentage_missing(series):
   ''' Calculates percentage of NaN values in DataFrame
   :param series: Pandas DataFrame object
   :return: float
   '''
   num = series.isnull().sum()
   den = len(series)
   return round(num/den, 2)# Only include columns that contain any NaN values

In [62]:
get_percentage_missing(survey_place_less)

arr_time                   0.00
dep_time                   0.00
trip_distance_miles        0.24
air_trip_distance_miles    0.24
prev_trip_duration_min     0.24
act_dur                    0.00
act_cnt                    0.00
tract_id                   0.00
county_id                  0.00
state_id                   0.00
place_primarycity          0.00
city                       0.00
state                      0.00
zipcode                    0.00
mode                       0.24
dtype: float64

In [63]:
survey_place_less.dropna(inplace=True)

/home/g49/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
get_percentage_missing(survey_place_less)

arr_time                   0.0
dep_time                   0.0
trip_distance_miles        0.0
air_trip_distance_miles    0.0
prev_trip_duration_min     0.0
act_dur                    0.0
act_cnt                    0.0
tract_id                   0.0
county_id                  0.0
state_id                   0.0
place_primarycity          0.0
city                       0.0
state                      0.0
zipcode                    0.0
mode                       0.0
dtype: float64

In [65]:
y = survey_place_less.pop('mode')

In [66]:
dict_not_transit = dict((key, 0) for key in range(15))
dict_transit = dict((key, 1) for key in range(15,30))
mode_dict = {**dict_not_transit, **dict_transit}

In [70]:
ymap = y.map(mode_dict)

In [71]:
print(f"{ymap[ymap>0].count()} of {len(ymap)} ({ymap[ymap>0].count()/len(ymap)*100:.2f}%) are public transit.")

12816 of 349300 (3.67%) are public transit.


In [72]:
X_train, X_test, y_train, y_test = train_test_split(survey_place_less, ymap)

In [73]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train);

ValueError: could not convert string to float: 'E4P 1'

In [ ]:
gbc.score(X_test, y_test)

In [74]:
abc = AdaBoostClassifier()
abc.fit(X_train, y_train);

ValueError: could not convert string to float: 'E4P 1'

In [ ]:
abc.score(X_test, y_test)

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);

In [ ]:
rfc.score(X_test, y_test)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
gbc2 = GradientBoostingClassifier()
gbc2.fit(X_train_scaled, y_train);

In [ ]:
gbc2.score(X_test, y_test)

In [ ]:
abc2 = AdaBoostClassifier()
abc2.fit(X_train_scaled, y_train);

In [ ]:
abc2.score(X_test, y_test)

In [ ]:
rfc2 = RandomForestClassifier()
rfc2.fit(X_train_scaled, y_train);

In [ ]:
rfc2.score(X_test, y_test)